# Block 6 Exercise 2: finding the best parameters for predicting the fare of taxi rides
We return to our Random Forest Regression and want to automatically optimize all free parameters ...

In [1]:
import pandas as pd
import numpy as np
import folium


In [2]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv('../../DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)

### Scikit Optimize
Scikit Optimize (https://scikit-optimize.github.io/stable/index.html) is a AutoML toolbox wrapped around Scikit-Learn. It allows us to use state-of-the-art automatic hyper-parameter optimization on top of our learning algorithms.   



In [3]:
# install 
!pip install scikit-optimize

### E 2.1 Bayesian Optimization of a Random Forest Regression Model
use Bayesian Optimization with Cross-Validation (https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV) to find the best regression model. Compare
* linear regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) 
* Random Forest regression (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* and SVM regression (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)

NOTES: this can become quite compute intensive! Hence,
* use a smaller subset of the training data to run the experiments 
* think about the range of your parameters (e.g. larger number of trees in RF or high C-values in SMV will make models expensive)
* optimize only the following parameters per model type:
    * linear: no parameters to optimize
    * RF: #trees and depth
    * SVM: C and gamma (use RBF kernel)
* parallelize -> n_jobs
* use CoLab to rum the job for up to 12h 


In [4]:
#split in train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.to_numpy()[ :10000, :], y[: 10000])

In [30]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.svm import SVC
from sklearn.svm import SVR

In [48]:
opt = BayesSearchCV(
    SVR(),
    {
        'C': Real(1e-6, 1e+6, prior='log-uniform'),
        'gamma': Real(1e-6, 1e+1, prior='log-uniform'),

    },
    cv=5,
     n_iter=32,
    random_state=0,
    n_jobs=4, #parallelize
    scoring='neg_mean_squared_error'
)

In [49]:
opt.fit(X_train, y_train)

BayesSearchCV(cv=5, estimator=SVR(), n_iter=32, n_jobs=4, random_state=0,
              scoring='neg_mean_squared_error',
              search_spaces={'C': Real(low=1e-06, high=1000000.0, prior='log-uniform', transform='identity'),
                             'gamma': Real(low=1e-06, high=10.0, prior='log-uniform', transform='identity')})

In [50]:
print(opt.score(X_test, y_test))

-20.436487280387446


In [51]:
opt.best_score_

-19.127946079525046

In [38]:
from sklearn.linear_model import LinearRegression

In [43]:
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_test, y_test)

0.7109964017635311

In [34]:
from sklearn.ensemble import RandomForestRegressor

In [35]:
rf = BayesSearchCV( 
         RandomForestRegressor(),
         {
             'n_estimators':Integer(10,100, prior='log-uniform'),
             'max_depth':Integer(2,10,prior='log-uniform')
         },
         n_iter=32,
         random_state=0,
         n_jobs=4, #parallelize
         cv=5, # set to 5 folds
         scoring='neg_mean_squared_error' #eval MSE as before -> see https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter for other scoring methods
     )

In [36]:
rf.fit(X_train, y_train)

C:\Users\Lena Huber\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\Lena Huber\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\Lena Huber\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\Lena Huber\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\Lena Huber\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been ev

BayesSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=32, n_jobs=4,
              random_state=0, scoring='neg_mean_squared_error',
              search_spaces={'max_depth': Integer(low=2, high=10, prior='log-uniform', transform='identity'),
                             'n_estimators': Integer(low=10, high=100, prior='log-uniform', transform='identity')})

In [37]:
print(rf.score(X_test, y_test))

-16.151690147593495
